In [2]:
import sys

import time
import numpy as np
import sys,os
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from hfnet.settings import EXPER_PATH
from notebooks.utils import plot_images, plot_matches, add_frame

import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
import logging
from tqdm import tqdm
import numpy as np
from scipy.spatial.transform import Rotation as R, Slerp 
from numpy.linalg import inv, norm
logging.basicConfig(format='[%(asctime)s %(levelname)s] %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
from hfnet.datasets import get_dataset  # noqa: E402
from hfnet.evaluation.loaders import export_loader  # noqa: E402
from hfnet.settings import EXPER_PATH  # noqa: E402
from scipy import spatial


tf.contrib.resampler  # import C++ op
class HFNet:
    def __init__(self, model_path, outputs):
        self.session = tf.Session()
        self.image_ph = tf.placeholder(tf.float32, shape=(None, None, 3))

        net_input = tf.image.rgb_to_grayscale(self.image_ph[None])
        tf.saved_model.loader.load(
            self.session, [tag_constants.SERVING], str(model_path),
            clear_devices=True,
            input_map={'image:0': net_input})

        graph = tf.get_default_graph()
        self.outputs = {n: graph.get_tensor_by_name(n+':0')[0] for n in outputs}
        self.nms_radius_op = graph.get_tensor_by_name('pred/simple_nms/radius:0')
        self.num_keypoints_op = graph.get_tensor_by_name('pred/top_k_keypoints/k:0')
    def inference(self, image, nms_radius=4, num_keypoints=1000):
        inputs = {
            self.image_ph: image[..., ::-1].astype(np.float),
            self.nms_radius_op: nms_radius,
            self.num_keypoints_op: num_keypoints,
        }
        return self.session.run(self.outputs, feed_dict=inputs)

model_path = Path(EXPER_PATH, 'saved_models/hfnet')
outputs = ['global_descriptor', 'keypoints', 'local_descriptors','scores']
hfnet = HFNet(model_path, outputs)

/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWa

In [4]:
%load_ext autoreload
%autoreload 2
db_path =  "/home/pang/colmap_ws/office/office-1-5/sfm1/"
query_path ="/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/" 

db_list = os.listdir(db_path)
query_list = os.listdir(query_path)
query_list.sort()
db_list.sort()

db_image = lambda n: cv2.imread(db_path + n)[:, :, ::-1]
query_image = lambda n: cv2.imread(query_path + n)[:, :, ::-1]
images_db = [db_image(frame) for frame in db_list]
images_query = [query_image(frame) for frame in query_list]

db = [hfnet.inference(i) for i in images_db]
global_index = np.stack([d['global_descriptor'] for d in db])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
## export feature for SfM
export_path = '/home/pang/data/dataset/iros2019slam/office/office-1-1/hf_feat/'
if not os.path.isdir(export_path):
    os.mkdir(export_path)


for i in range(len(images_db)):
    data = hfnet.inference(images_db[i])
    export = {
        'keypoints': data['keypoints'],
        'local_descriptors': data['local_descriptors'],
        'global_descriptor':data['global_descriptor'],
        'scores':data['scores']
    }
    name = export_path + db_list[i][:-4]
    np.savez(f'{name}.npz', **export)

In [6]:
print(db[0])

{'global_descriptor': array([ 0.04144748, -0.03778127, -0.01158974, ...,  0.00468009,
       -0.01940508,  0.00099139], dtype=float32), 'keypoints': array([[127, 378],
       [ 48, 362],
       [452, 353],
       ...,
       [ 15, 280],
       [759,  29],
       [724, 233]], dtype=int32), 'local_descriptors': array([[-0.11879543,  0.05730081, -0.11559588, ..., -0.06991927,
        -0.08719563, -0.03810052],
       [-0.0304197 ,  0.14113088,  0.01225519, ...,  0.01745378,
         0.02705371,  0.10619936],
       [ 0.07104245, -0.00240788, -0.04233182, ..., -0.0680691 ,
        -0.00735071, -0.00285753],
       ...,
       [-0.06359141,  0.04361811, -0.03814925, ...,  0.03785016,
        -0.01573319, -0.04820301],
       [ 0.02921025, -0.03192076,  0.09027757, ..., -0.21929972,
        -0.04235946,  0.00830175],
       [ 0.03126141, -0.02725485,  0.00025875, ...,  0.04880438,
        -0.01269413,  0.04172838]], dtype=float32), 'scores': array([0.47909668, 0.4307392 , 0.4150982 , 0.40706